# Import libraries:

In [1]:
import os
import sys
import re
import time
import string

import requests
import sys
import time

import pprint
pp = pprint.PrettyPrinter(indent=4)

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('stopwords')
from nltk.corpus import stopwords
eng_stop_words = set(stopwords.words('english'))
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from pyinflect import getAllInflections, getInflection
#from pymystem3 import Mystem
#m = Mystem()
from nltk.stem.snowball import SnowballStemmer 
ru_stemmer = SnowballStemmer("russian") 
from nltk.stem import WordNetLemmatizer 
english_lemmatizer = WordNetLemmatizer()

import requests
from bs4 import BeautifulSoup
import urllib
import urllib.request
from wiktionaryparser import WiktionaryParser
import json
#from google.cloud import translate
# GOOGLE_APPLICATION_CREDENTIALS="/Users/markdecourcyling/Desktop/Translating_collocations.json"
#translate_client = translate.Client.from_service_account_json(support_files_dir + '/Translating_collocations.json')

import pandas as pd
import numpy as np
from translate.storage.tmx import tmxfile
import seaborn as sns

import genanki

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MdeCL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MdeCL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MdeCL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# FUNCTIONS:

In [2]:
def monoling_wikt_scrape(word):
    url = "https://en.wiktionary.org/w/index.php?title=" + word + '&printable=yes'
    response = requests.get(url)
    wikt_definitions = []
    try:
        soup = BeautifulSoup(response.text, "html.parser")
        if soup.find('ol') != None:
            entries = soup.find('ol')
            #if entries.findAll('li') != None:
            if entries.select('ol > li') != None:
                #number_of_defs = len(entries.findAll('li'))
                number_of_defs = len(entries.select('ol > li'))
                index = 0
                while index + 1 <= number_of_defs:
    
                    # definition:
                    entry = entries.select('ol > li')[index]
                    entry_text = entry.get_text()

                    if entry_text != '':

                        # register tag:           
                            
                        if entry.find('ul') != None:
                            sent_info_tag = entry.find('ul')
                            sent_info_text = sent_info_tag.get_text()
                            entry_text = entry_text.replace(sent_info_text, '')
                            
                            
                        ex_sent_text = ''
                        if entry.find('dd') != None:
                            ex_sent_tag = entry.find('dl')
                            ex_sent_text = ex_sent_tag.get_text()
                            entry_text = entry_text.replace(ex_sent_text, '')

                        capitalised_form = word.replace(word[0], word[0].upper())

                        ex_sent_with_blank = ex_sent_text.replace(word, '_____')
                        ex_sent_with_blank = ex_sent_with_blank.replace(capitalised_form, '_____')
                        ex_sent_full = ex_sent_text.replace(word, '<b>' + word + '</b>')
                        ex_sent_full = ex_sent_text.replace(capitalised_form, '<b>' + capitalised_form + '</b>')

                        definition_package = {'definition' : entry_text,
                                              'ex_sentence_blank' : ex_sent_with_blank,
                                                'ex_sentence_full' : ex_sent_full
                                             }

                        wikt_definitions.append(definition_package)
                    
                    index += 1
                    
                return(wikt_definitions)
            else:
                return('Error')
        else:
            return('Error')
    except Exception as e:
        return e

In [3]:
monoling_wikt_scrape('homogolation')

'Error'

In [4]:
def synset_syns(word):
    syn_results = wordnet.synsets(word)
    syns = []
    for syn in syn_results:
        for lm in syn.lemmas():
            syn_word = lm.name()
            syns.append(syn_word)
    return syns

In [5]:
synset_syns('cataclysmic')

['cataclysmal', 'cataclysmic']

In [6]:
def skell_synonyms_scrape(word):
    base_url = "https://skell.sketchengine.co.uk/run.cgi/thesaurus?lpos=&query="
    response = requests.get("{url}/{endpoint}".format(url=base_url, endpoint=word))
    
    syns = []
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        if soup.find('div', {'id' : 'thes_content'}) != None:
            thes_content = soup.find('div', {'id' : 'thes_content'})
            
            syns = []
            
            syn_idx = 0
            
            while syn_idx + 1 <= 6:
                
                try:
                    if thes_content.findAll('li')[syn_idx].get_text() != None:
                        syn = thes_content.findAll('li')[syn_idx].get_text()
                        syns.append(syn)
                except:
                    pass
                
                syn_idx += 1
        else:
            synonyms_text = "No synonyms found."
    else:
        print('\tSynonym scraper couldn\'t connect to Skell for ' + word + '.')
        
    return(syns)

In [7]:
skell_synonyms_scrape('mimetic')

[]

In [8]:
def skell_example_sentence_scrape(word):
    base_url = "https://skell.sketchengine.co.uk/run.cgi/concordance?lpos=&query="
    response = requests.get("{url}/{endpoint}".format(url=base_url, endpoint=word))
    print(response)
    soup = BeautifulSoup(response.text, "html.parser")
    
    ex_sents_blank = ''
    ex_sents_full = ''
    content_div = soup.find('div', {'id' : 'content'})
    if content_div.find('tr') != None:
        for tr_div in content_div.find_all('tr', limit = 4):
            for sent_div in tr_div.find_all('td'):
                if sent_div.has_attr("class") == True:
                    pass
                else:
                    if sent_div.find('span', {'class':'lc'}) != None:
                        pre_term = sent_div.find('span', {'class':'lc'})
                        pre_term_text = pre_term.get_text()
                    else:
                        pre_term_text = ''
                        
                    if sent_div.find('span', {'class':'kw'}) != None:
                        term = sent_div.find('span', {'class':'kw'})
                        term_text = term.get_text().strip()
                        
                    if sent_div.find('span', {'class':'rc'}) != None:
                        post_term = sent_div.find('span', {'class':'rc'})
                        post_term_text = post_term.get_text()
                    else:
                        post_term_text = ''
                    ex_sent_blank = pre_term_text + ' _______ ' + post_term_text
                    ex_sent_full = pre_term_text + ' <b>' + term_text + '</b>' + post_term_text

                    ex_sents_blank += ex_sent_blank + '<br><br>'
                    ex_sents_full += ex_sent_full + '<br><br>'
            
    else:
        example_sentence_text = "No example sentence found."

    return{'Example sentences with blanks' : ex_sents_blank,
           'Example sentences full' : ex_sents_full
           }

In [10]:
skell_example_sentence_scrape('loss')

<Response [200]>


AttributeError: 'NoneType' object has no attribute 'find'

In [11]:
def skell_ex_sents_scrape(word):
    
    base_url = "https://skell.sketchengine.co.uk/run.cgi/thesaurus?lpos=&query="
    response = requests.get("{url}/{endpoint}".format(url=base_url, endpoint=word))
    
    ex_sents_blank = []
    ex_sents_full = []
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
    
    return {'Example sentences with blanks' : ex_sents_blank,
           'Example sentences full' : ex_sents_full}

In [13]:
skell_ex_sents_scrape('loss')

{'Example sentences with blanks': [], 'Example sentences full': []}

# Write the flashcards:

In [14]:
note_type = genanki.Model(
  1603245899,
  'Monolingual English Vocab Test Note Type',
  fields = [
    {'name': 'Term'},
    {'name': 'First letter'},
    {'name' : 'Wikt defs blanks'},
    {'name' : 'Wikt defs full'},
    {'name' : 'Syns'},
    {'name': 'Ex sents with blanks'},
    {'name': 'Ex sents full'}
  ],
  templates = [
    {
      'name': 'Monolingual English Vocab Test Card',
      'qfmt': '<font color="navy" size="+2">{{Syns}}</font><br><hr><font size="+1">{{Ex sents with blanks}}<br>{{type:Term}}</font><hr><i><font color="firebrick">{{Wikt defs blanks}}<br><br><br><br><br><br><br><br><br><br><font size="-3">First letter:<br>{{First letter}}</font></i>',
      'afmt': '<font color="navy" size="+2">{{Syns}}</font><br><hr><font size="+1">{{Ex sents full}}<br>{{type:Term}}</font><hr><i><font color="firebrick">{{Wikt defs blanks}}<br><br><br><br><br><br><br><br><br><br><font size="-3">First letter:<br>{{First letter}}</font></i>'
    }
  ]
#     css='.card{font-family: arial;font-size: 20px;text-align: left;color: black;background-color: #88FFDD;}',
)

In [15]:
my_deck = genanki.Deck(
  2059400110,
  'English Vocab Deck')

word_list = []
with open('C://Users//MdeCL//OneDrive//English Vocab Project//business_vocab.txt', 'r') as input_file:
    word_list = input_file.readlines()
    word_list = [item.replace('\n', '').lower() for item in word_list]

word_list = list(set(word_list))

In [16]:
idx = 1
for term in word_list:
    if term != '':
        print(idx)
        print(term)
        try:
            
            first_letter = term[0]
            
            start_time = time.time()
            
            wikt_defs = monoling_wikt_scrape(term)
            wikt_defs_blanks = ''
            wikt_defs_full = ''
            for wikt_def in wikt_defs:
                wikt_defs_blanks += '<font size="+2">' + wikt_def['definition'] + '<br></font>'
                wikt_defs_full += '<font size="+2">' + wikt_def['definition'] + '<br></font>'
                wikt_defs_blanks += wikt_def['ex_sentence_blank'] + '<br><br>'
                wikt_defs_full += wikt_def['ex_sentence_full'] + '<br><br>'

            syns = ', '.join(skell_synonyms_scrape(term))

            ex_sents = skell_example_sentence_scrape(term)
            ex_sents_blanks = ex_sents['Example sentences with blanks']
            ex_sents_full = ex_sents['Example sentences full']

            note = genanki.Note(
                            model = note_type,
                            fields = [term,
                                      first_letter,
                                      wikt_defs_blanks,
                                      wikt_defs_full,
                                      syns,
                                      ex_sents_blanks,
                                      ex_sents_full
                                        ]
                            )

            my_deck.add_note(note)
            idx += 1
            
            end_time = time.time()
            time_taken = end_time - start_time
            if time_taken < 3:
                time.sleep(3 - time_taken)
    
        except Exception as e:
            print(e)

genanki.Package(my_deck).write_to_file('C:\\Users\\MdeCL\\Desktop\\monoling_eng_vocab.apkg')
print('\n\nFlashcard writing complete.')

1
feasibility study
string indices must be integers
1
redouble
<Response [200]>
'NoneType' object has no attribute 'find'
1
remuneration
<Response [200]>
'NoneType' object has no attribute 'find'
1
homogolation
string indices must be integers
1
run aground
<Response [200]>
'NoneType' object has no attribute 'find'
1
horizontal integration
<Response [200]>
'NoneType' object has no attribute 'find'
1
continuity plans
string indices must be integers
1
c-suite
string indices must be integers
1
market penetration
string indices must be integers
1
e-commerce
<Response [200]>
'NoneType' object has no attribute 'find'
1
distribution partner
string indices must be integers
1
data lake
<Response [200]>
'NoneType' object has no attribute 'find'
1
ideate
<Response [200]>
'NoneType' object has no attribute 'find'
1
poised
<Response [200]>
'NoneType' object has no attribute 'find'
1
business model
<Response [200]>
'NoneType' object has no attribute 'find'
1
upend
<Response [200]>
'NoneType' object h

In [17]:
genanki.Package(my_deck).write_to_file('C:\\Users\\MdeCL\\Desktop\\monoling_eng_vocab.apkg')

In [ ]:
with open('/Users/markdecourcyling/Desktop/english_lit_vocab_flashcards.txt', 'w+') as file:
    with open('/Users/markdecourcyling/Desktop/english_lit_vocab.txt', 'r') as word_list:
        word_list_read = word_list.read()
        word_list = word_list_read.split('\n')
    
    word_list = list(set(word_list))
    language = 'english'
    parser = WiktionaryParser()
    for word in word_list:
        word = word.lower()
        if parser.fetch(word, language) != None:
            if parser.fetch(word, language):
                try:
                    data = parser.fetch(word, language)[0]
                    #time.sleep(2)
                    definitions = data['definitions'][0]['text']
                    item_idx = 0
                    def_list = definitions[1:]
                    for definition in def_list:
                        definitions_with_blank = ''
                        definitions_full = ''
                        for item in def_list:
                            if definition == item:
                                definitions_with_blank += '<b>' + '???????' + '</b><br><br>'
                                definitions_full += '<b>' + item + '</b><br><br>'
                            else:
                                definitions_with_blank += item + '<br><br>'
                                definitions_full += item + '<br><br>'
                        print(word + '£<br>' + definitions_with_blank + '£<br>' + definitions_full + '\n')
                        file.write(word + '£<br>' + definitions_with_blank + '£<br>' + definitions_full + '\n')
                        test_recall_flashcards.write(word + '£<br>' + definitions_full + '\n')
                        item_idx += 1
                except IndexError:
                    print('COULDN\'T WRITE CARDS FOR:')
                    print(word)
                    pass
            else:
                continue
        else:
            continue

In [ ]:
with open('/Users/markdecourcyling/Desktop/english_lit_vocab_flashcards.txt', 'w+') as file:
    
    with open('/Users/markdecourcyling/Desktop/english_lit_vocab.txt', 'r') as word_list:
        word_list_read = word_list.read()
        word_list = word_list_read.split('\n')
    
    for word in word_list:
        dict_defs = monoling_wikt_scrape(word)
        
        print(dict_defs)
        
        if dict_defs != 'Error':

            item_idx = 0
            for dict_definition in dict_defs:
                
                monoling_defs_blank = ''
                monoling_defs_complete = ''
                
                for item in dict_defs:

                    definition = dict_definition['definition']
                    ex_sent_blank = dict_definition['ex_sentence_blank']
                    ex_sent_full = dict_definition['ex_sentence_full']

                    if dict_defs.index(item) == item_idx:
                        monoling_defs_blank += '<b>' + '???????' + '<br><emsp>' + ex_sent_blank + '</b><br><br>'

                        monoling_defs_complete += '<b>' + definition + '<br><emsp>' + ex_sent_blank + '</b><br><br>'

                    else:
                        monoling_defs_blank += definition + '<br><emsp>' + ex_sent_blank + '<br><br>'

                        monoling_defs_complete += definition + '<br><emsp>' + ex_sent_full + '<br><br>'


                    print(word + '£<br>' + monoling_defs_blank + '£<br>' + monoling_defs_complete + '\n')

                    file.write(word + '£<br>' + monoling_defs_blank + '£<br>' + monoling_defs_complete + '\n')

                    item_idx += 1
    
monoling_wikt_scrape('admonition')

In [ ]:
def wiktionary_parser_eng_trans_scrape(word):
    language = 'english'

    parser = WiktionaryParser()
    if parser.fetch(word, language) != None:
        if parser.fetch(word, language) != None:
            data = parser.fetch(word, language)
            try:
                definitions = data[0]['definitions']  
                try:
                    definitions_list = definitions[0]['text']
                    definitions_list = definitions_list[1:]
                    for definition in definitions_list:
                        if 'Alternative spelling of ' in definition:
                            alt_spelling = definition.split('Alternative spelling of ')[1]
                            # print(alt_spelling.split())
                            # print(alt_spelling.split()[0])
                            alt_spelling = alt_spelling.split()[0]
                            # print(alt_spelling)
                            if parser.fetch(alt_spelling, language) != None:
                                    data = parser.fetch(alt_spelling, language)
                                    definitions = data[0]['definitions']
                                    print('Wiktionary scraper found the alt spelling \'' + alt_spelling + '\' and returned its definitions.')
                                    if definitions:
                                        definitions_list = definitions[0]['text']
                                        return_list = definitions_list[1:]
                                    else:
                                        return_list = ['WiktionaryParser can\'t find definitions for \'' + alt_spelling + '\'.']
                        else:
                            return_list = definitions_list
                            
                except Exception:
                    return_list = ['No', 'translations', 'found']
            except Exception:
                return_list = ['No', 'translations', 'found']
        else:
            # complete_definitions_text = 'WiktionaryParser can\'t find definitions for \'' + word + '\'.'
            return_list = ['No', 'translations', 'found']
    else:
        # complete_definitions_text = 'WiktionaryParser can\'t find definitions for \'' + word + '\'.'
        return_list = ['No', 'translations', 'found']
    
    return(return_list)

# Edit the flashards to make more guessable:

In [ ]:
with open('C:\\Users\\MdeCL\\OneDrive\\English Vocab Project\\1 - English Writing Skills__English Vocab (Early Easter Vac 2020).txt', 'r', encoding="utf8") as file:
    with open('C:\\Users\\MdeCL\\OneDrive\\English Vocab Project\\1 - English Writing Skills__English Vocab (Early Easter Vac 2020) 2.txt', 'w', encoding="utf8") as output_file:
        
        for line in file.readlines():

            line_list = line.split('\t')
            
            first_letter = line_list[0][0]
            # PUT THE FIRST LETTER IN FRONT OF EACH BLANK
#             line_list = [re.sub(r'_+', first_letter + '______', item) for item in line_list]
            
            # PUT THE FIRST LETTER AT THE BOTTOM OF THE CARD, SO I HAVE TO SCROLL TO SEE IT.
            line_list[1] = line_list[1] + '<br><br><br><br><br><br><br><br><br><br><br>' + 'First letter:' + '<br>' + first_letter

            output_file.write('\t'.join(line_list) + '\n')